<a href="https://colab.research.google.com/github/ChrisFJardine/Masterclass/blob/main/CJardineAssignment3__Webcrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re
import time
import os
from datetime import datetime, timedelta

In [2]:
#Map Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def get_date(soup):                  #Gets the date from the page - superceded 
  mydate = soup.findAll("div", {"class": "form-group has-feedback date"})
  for x in mydate:
    children = x.findChildren("input" , recursive=False)
    for child in children:
      date = child['value']
  return(date)

In [4]:
def get_previous_date(soup):           #Gets the date from the 'previous' input button 
  link = soup.findAll("a", {"class": "prev"}, href = True)
  for x in link:
    prev_date = (x['href'].replace('?date=', ''))   #'back date' button
  return(prev_date)

In [5]:
def get_news(soup):                   #extracts the teaser text from a page -class comp-rich-text teaser-text clearfix
  mydivs = soup.findAll("div", {"class": "comp-rich-text teaser-text clearfix"})
  ignore = "ABC News"
  news_item = []
  news = []
  date = get_date(soup)
  for div in mydivs:
    children = div.findChildren("p" , recursive=False)    #Get data from p divs
    for child in children:
      if ignore not in child.get_text():
        news_item = [date, child.get_text()]              #extract the teaser text and associated with the date
        news.append(news_item)                            #build the list.
      else:
        continue
  return(news)

In [7]:
#Crawl web page archives and extract soup
urlbase = "https://www.abc.net.au/news/archive/?date="
date = '2021-11-27'                                       #Starts here and crawls across historical dates
news = []
print('Launching Webcrawler on ' + urlbase + date)
for i in range(4000):
  url = urlbase + date                                    #page to pull
  try:
    page = requests.get(url)
    print('Request #' + str(i) + ' : retrieved in  ' + str(page.elapsed.total_seconds()) + ' seconds')
  except:
    print('No soup for you! ' + str(page.status_code))   
    date = datetime.strptime(date, '%Y-%m-%d') - timedelta(1) #failed, move on to next day
  if len(page.content) > 1000:
    soup = BeautifulSoup(page.text, 'lxml')               
    news.append(get_news(soup))                           #Get the news, build the list of articles
    date = get_previous_date(soup)                        #Get the date for the next page
    time.sleep(0.5)                                       #Pad request to avoid being a pest

news = pd.DataFrame(news)
#news.to_csv('/content/drive/MyDrive/Colab Notebooks/Assignment 3/ExtractedNews.csv', index=True) 

Launching Webcrawler on https://www.abc.net.au/news/archive/?date=2021-11-27
Request #0 : retrieved in  0.011681 seconds
Request #1 : retrieved in  0.012189 seconds
Request #2 : retrieved in  0.015284 seconds
Request #3 : retrieved in  0.011292 seconds
Request #4 : retrieved in  0.013421 seconds
Request #5 : retrieved in  0.294537 seconds
Request #6 : retrieved in  0.428726 seconds
Request #7 : retrieved in  0.014552 seconds
Request #8 : retrieved in  0.0133 seconds
Request #9 : retrieved in  0.01848 seconds
